In [1]:
import sys
import os
import numpy as np
import pyvista as pv
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
from helpers import safe_parse_quantity, delete_pyvista_fields, map_on_control_mesh
import pint
import vtk
from tqdm import tqdm

In [2]:
# Add it to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.helper import calculate_normal

In [3]:
my_cmap = plt.get_cmap("coolwarm", 15)
root = Path().cwd().parent / "Snapshots"
print(root)
print(root.exists())

/Users/thomassimader/Documents/NIRB/Snapshots
True


In [4]:
dx = 50
dy = 10

alpha_1 = np.arctan(dy / dx)
alpha_2 = np.arctan(2*dy / dx)
alpha_3 = np.arctan(dy / (2 * dx))

print(f"alpha_1 = {np.rad2deg(alpha_1)}")
print(f"alpha_2 = {np.rad2deg(alpha_2)}")
print(f"alpha_3 = {np.rad2deg(alpha_3)}")
print(f"alpha_2 - alpha_1 = {np.rad2deg(alpha_2 - alpha_1)}")
print(f"alpha_1 - alpha_3 = {np.rad2deg(alpha_1 - alpha_3)}")


alpha_1 = 11.309932474020215
alpha_2 = 21.80140948635181
alpha_3 = 5.710593137499643
alpha_2 - alpha_1 = 10.491477012331599
alpha_1 - alpha_3 = 5.599339336520571


In [5]:
PARAMETER_SPACE = "01"
DATA_TYPE = "Test"

In [6]:

path_domain_3d = root / PARAMETER_SPACE / "Training" / "Training_001.vtu"
domain_3d = COMSOL_VTU(path_domain_3d)
print(domain_3d.mesh.bounds)

(0.0, 4000.0000000000005, 0.0, 5000.000000000002, -4000.000000000001, 0.0)


In [7]:
def create_smooth_bounds(mesh: pv.DataSet, val_old : float, val_new: float, axis: int = 2) -> pv.DataSet:
    mesh.points[:, axis] = np.where(mesh.points[:, axis] == val_old, val_new, mesh.points[:, axis])
    return mesh

print(domain_3d.mesh.bounds)#
bound_indices = [1, 3, 4]
for i_axis, i_bound in enumerate(bound_indices):
    while np.abs(domain_3d.mesh.bounds[i_bound]) > int(np.abs(domain_3d.mesh.bounds[i_bound])):
        domain_3d.mesh = create_smooth_bounds(domain_3d.mesh,
                                              domain_3d.mesh.bounds[i_bound],
                                              int(domain_3d.mesh.bounds[i_bound]) , axis=i_axis)
print(domain_3d.mesh.bounds)

(0.0, 4000.0000000000005, 0.0, 5000.000000000002, -4000.000000000001, 0.0)
(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)


In [8]:

# destination_mesh = domain_3d.mesh
# data_folder = Path(root / VERSION / DATA_TYPE)
# assert data_folder.exists(), f"Data folder {data_folder} does not exist."
# export_folder =  data_folder.parent / "Truncated"
# export_folder.mkdir(exist_ok=True)
# assert export_folder.exists(), f"Export folder {export_folder} does not exist."
# vtu_files = sorted([path for path in data_folder.iterdir() if path.suffix == ".vtu"])
# for idx, vtu_file in tqdm(enumerate(vtu_files), total=len(vtu_files), desc="Reading COMSOL files"):
#     temp_data = COMSOL_VTU(vtu_file)
#     temp_data = delete_pyvista_fields(temp_data, ["Temperature"])
#     mapped = map_on_control_mesh(temp_data.mesh, destination_mesh)
#     mapped.save(export_folder / vtu_file.name)
# destination_mesh.points.shape

In [9]:
bounds = domain_3d.mesh.bounds  # (xmin, xmax, ymin, ymax, zmin, zmax)

# Grid resolution
nx, ny, nz = 100, 100, 100  # You can increase this for more resolution

dx, dy, dz = 100, 100, 100  # You can increase this for more resolution
# Compute the number of points needed
nx = int((bounds[1] - bounds[0]) / dx) + 1
ny = int((bounds[3] - bounds[2]) / dy) + 1
nz = int((bounds[5] - bounds[4]) / dz) + 1


image = vtk.vtkImageData()
image.SetDimensions(nx, ny, nz)
image.SetOrigin(bounds[0], bounds[2], bounds[4])
image.SetSpacing(dx, dy, dz)
# image.SetSpacing(
#     (bounds[1] - bounds[0]) / (nx - 1),
#     (bounds[3] - bounds[2]) / (ny - 1),
#     (bounds[5] - bounds[4]) / (nz - 1),
# )

interpolated_vtk = map_on_control_mesh(domain_3d.mesh, image)
print(bounds)
print(f"{(bounds[1] - bounds[0]) / (nx)}" )
# pv_data = pv.wrap(image).save("interpolated_output.vti")

(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
97.5609756097561


In [14]:

path_domain_parameters = root /  PARAMETER_SPACE / "Exports" / f"{path_domain_3d.stem}_parameters.csv"
assert path_domain_parameters.exists()
df = pd.read_csv(path_domain_parameters, index_col = 0)
ureg = pint.UnitRegistry()
df['quantity_pint'] = df[df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
# df

In [13]:
domain_3d.info()
fields = domain_3d.exported_fields.copy()
fields.remove("Temperature")
for field in fields:
    domain_3d.delete_field(field)

self.vtu_path=PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Training/Training_001.vtu')
41 timesteps from 0.000e+00 s to 4.000e+13 s
self.mesh.bounds=(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
Availabe fields in vtu dataset:
	 1: Dynamic_viscosity
	 2: Fluid_density
	 3: Permeability,_XX-component
	 4: Permeability,_XY-component
	 5: Permeability,_XZ-component
	 6: Permeability,_YY-component
	 7: Permeability,_ZZ-component
	 8: Pressure
	 9: Temperature
	 10: Total_Darcy_velocity_magnitude


In [15]:
dip    = df.loc["dip", "quantity_pint"].to("deg")
strike = df.loc["strike", "quantity_pint"].to("deg")
print(dip)
print(strike)
print(np.round(dip.magnitude))
print(np.round(strike.magnitude))

59.99999999999999 degree
90.0 degree
60.0
90.0


### Original Data

In [16]:
field_name = domain_3d.format_field("Temperature", -1)
normal_vector = calculate_normal(np.round(dip.magnitude), np.round(strike.magnitude))
clip_original = domain_3d.mesh.clip(normal = -normal_vector, origin=domain_3d.mesh.center)
# clip_original.plot(scalars = field_name, cmap=my_cmap)


### Control Mesh

In [33]:
bounds = domain_3d.mesh.bounds
dx, dy, dz = (100, 100, 100)  # You can increase this for more resolution
# Compute the number of points needed (+ 1 as spacing is an interval)
nx = int(np.abs((bounds[1] - bounds[0])) / dx) + 1
ny = int(np.abs((bounds[3] - bounds[2])) / dy) + 1
nz = int(np.abs((bounds[5] - bounds[4])) / dz) + 1
grid = pv.ImageData() 
grid.origin = (bounds[0], bounds[2], bounds[4])
grid.dimensions = np.array((nx, ny, nz))
grid.spacing = (dx, dy, dz)
grid.plot(show_edges=True)
# grid.slice_orthogonal().plot()
print(grid.bounds)
print(domain_3d.mesh.bounds)
print(grid)

Widget(value='<iframe src="http://localhost:53960/index.html?ui=P_0x32d069550_10&reconnect=auto" class="pyvist…

(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
(0.0, 4000.0, 0.0, 5000.0, -4000.0, 0.0)
ImageData (0x34755ce20)
  N Cells:      80000
  N Points:     85731
  X Bounds:     0.000e+00, 4.000e+03
  Y Bounds:     0.000e+00, 5.000e+03
  Z Bounds:     -4.000e+03, 0.000e+00
  Dimensions:   41, 51, 41
  Spacing:      1.000e+02, 1.000e+02, 1.000e+02
  N Arrays:     0


### Add plane

In [18]:
control_mesh = grid  #pv.merge([grid, clipped_plane])
control_mesh.clip(normal = -normal_vector, origin=domain_3d.mesh.center).plot()
control_mesh_cell = control_mesh.copy()


Widget(value='<iframe src="http://localhost:53960/index.html?ui=P_0x3115abbf0_1&reconnect=auto" class="pyvista…

### pv.interpolate()

vtkGaussianKernel is an interpolation kernel that simply returns the weights for all points found in the sphere defined by radius R. The weights are computed as: $e^{-(\frac{s*r}{R})^2}$ where r is the distance from the point to be interpolated to a neighboring point within R. The sharpness s simply affects the rate of fall off of the Gaussian. (A more general Gaussian kernel is available from vtkEllipsoidalGaussianKernel.)

https://vtk.org/doc/nightly/html/classvtkGaussianKernel.html

Examples
https://examples.vtk.org/site/Cxx/Meshes/InterpolateFieldDataDemo/

In [19]:
result              = control_mesh.interpolate(domain_3d.mesh, radius=500, sharpness=2, strategy="mask_points")
clip_interpolate = result.clip(normal = -normal_vector, origin=domain_3d.mesh.center)

# Visualize
# clip_interpolate.plot(scalars=field_name, cmap=my_cmap, show_edges=False)
# result.plot(scalars=field_name, cmap="coolwarm", show_edges=True)

### pv.Sample()

In [20]:
cell_mesh = domain_3d.mesh.point_data_to_cell_data(pass_point_data=False)
# control_mesh_cell.plot()
# Visualize
# clip.plot(scalars=field_name, cmap=my_cmap, show_edges=False)
# result.plot(scalars=field_name, cmap="coolwarm", show_edges=True)

In [21]:
result_cell              = control_mesh_cell.sample(domain_3d.mesh)
clip_sample = result_cell.clip(normal = -normal_vector, origin=domain_3d.mesh.center)

### vtkProbeFilter

In [32]:
bounds = domain_3d.mesh.GetBounds()  # (xmin, xmax, ymin, ymax, zmin, zmax)

dx, dy, dz = 50, 50, 25  # You can increase this for more resolution
# Compute the number of points needed
zmin = bounds[4] + dz
zmax = bounds[5] 

nx = int((bounds[1] - bounds[0]) / dx) + 1
ny = int((bounds[3] - bounds[2]) / dy) + 1
# nz = int((bounds[5] - bounds[4]) / dz) + 1
nz = int(np.abs((zmax - zmin)) / dz) + 1  # New z-range



image = vtk.vtkImageData()
image.SetDimensions(nx, ny, nz)
image.SetOrigin(bounds[0], bounds[2], zmin)
image.SetSpacing(dx, dy, dz)

interpolated_vtk = map_on_control_mesh(domain_3d.mesh, image)
# interpolated_vtk = map_on_control_mesh(interpolated_vtk, image)
# print(bounds)
# print(interpolated_vtk.bounds)
clip_vtk = interpolated_vtk.clip(normal = -normal_vector, origin=domain_3d.mesh.center)
clip_vtk.plot(scalars = field_name)
print(interpolated_vtk.bounds)


Widget(value='<iframe src="http://localhost:53960/index.html?ui=P_0x34a44b9b0_9&reconnect=auto" class="pyvista…

(0.0, 4000.0, 0.0, 5000.0, -3975.0, 0.0)


In [ ]:

producer = vtk.vtkTrivialProducer()
producer.SetOutput(domain_3d.mesh)  # .mesh is already a vtkDataSet

resample = vtk.vtkResampleToImage()
resample.SetInputConnection(producer.GetOutputPort())
resample.SetSamplingDimensions(nx, ny, nz)
resample.SetUseInputBounds(True)
resample.Update()

interpolated = pv.wrap(resample.GetOutput())
clip_vtk = interpolated.clip(normal = -normal_vector, origin=domain_3d.mesh.center)


## Plot

In [25]:

# Loop through the samples and plot
clips = [clip_original, clip_interpolate, clip_sample, clip_vtk]
labels = ["Original", "pv.Interpolate()", "pv.Sample()", "vtkProbeFilter"]
plotter = pv.Plotter(shape=(1, len(clips)), window_size=(1200, 500))
for i, (clip, label) in enumerate(zip(clips, labels)):
    plotter.subplot(0, i)
    plotter.add_mesh(clip, scalars = field_name,
                     cmap = my_cmap,
                    scalar_bar_args={'title': f'{field_name} ({i})',
                    'label_font_size': 10,
                    'title_font_size': 8,}
                    )
    

    plotter.add_text(label)

plotter.show()

Widget(value='<iframe src="http://localhost:53960/index.html?ui=P_0x312e06420_3&reconnect=auto" class="pyvista…